# * VINSIGHT : Metric Details

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Parameter Input '''

v_metric_cd = 'B0R000100GEO' #Total Revenue (Geo)
v_start_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_date: {v_start_date}')


Parameter input...

   -> v_metric_cd: B0R000100GEO
   -> v_start_date: 20250101


In [11]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Metric Details

In [3]:
''' Execute transaction '''


curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        , TARGET_SNAP, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = '{v_metric_cd}'
    AND TM_KEY_DAY >= {v_start_date}
    --ORDER BY TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Data as of 2025-05-29, 10:38:25


### Overview

In [15]:
''' ACTUAL : Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df['P_TARGET'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['TARGET_SNAP'], 0)
monthly_df['C'] = np.where(monthly_df['AREA_CD']=='C', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['P'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['G'] = np.where(monthly_df['AREA_TYPE']=='G', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['H'] = np.where(monthly_df['AREA_TYPE']=='H', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['HH'] = np.where(monthly_df['AREA_TYPE']=='HH', monthly_df['ACTUAL_SNAP'], 0)

monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,202501,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250131,"12,359,344,311",0,"12,034,347,430","12,018,244,403","12,017,986,832","12,017,979,514"
1,202502,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250228,"11,965,339,956",0,"11,619,121,062","11,576,992,535","11,577,051,847","11,577,036,719"
2,202503,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250331,"12,403,581,248",0,"11,995,980,677","11,934,524,905","11,932,828,205","11,932,819,678"
3,202504,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250430,"12,190,349,530",0,"11,891,585,207","11,881,040,681","11,878,822,083","11,878,641,780"
4,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,"10,771,445,001",0,"10,398,151,045","10,216,783,978","10,216,524,898","10,216,538,082"


In [16]:
''' ACTUAL : Daily Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

daily_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_tm_key_mth].copy()
daily_df['P_TARGET'] = np.where(daily_df['AREA_CD']=='P', daily_df['TARGET_SNAP'], 0)
daily_df['C'] = np.where(daily_df['AREA_CD']=='C', daily_df['ACTUAL_SNAP'], 0)
daily_df['P'] = np.where(daily_df['AREA_CD']=='P', daily_df['ACTUAL_SNAP'], 0)
daily_df['G'] = np.where(daily_df['AREA_TYPE']=='G', daily_df['ACTUAL_SNAP'], 0)
daily_df['H'] = np.where(daily_df['AREA_TYPE']=='H', daily_df['ACTUAL_SNAP'], 0)
daily_df['HH'] = np.where(daily_df['AREA_TYPE']=='HH', daily_df['ACTUAL_SNAP'], 0)

daily_df = daily_df.groupby(['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
daily_df = daily_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

daily_df_display = daily_df.copy()
mod_col_list = daily_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    daily_df_display[col] = daily_df_display[col].apply(lambda x: format(x, ',.0f'))

daily_df_display

,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P_TARGET,C,P,G,H,HH
0,20250501,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"163,008,010",0,"181,113,461","181,033,445","181,030,959","181,030,959"
1,20250502,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"1,008,924,196",0,"992,937,702","992,843,881","992,779,866","992,748,908"
2,20250503,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"237,739,560",0,"233,704,819","233,601,093","233,600,041","233,600,041"
3,20250504,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"234,556,209",0,"226,737,538","226,659,507","226,657,985","226,657,985"
4,20250505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"232,503,537",0,"229,099,212","229,023,042","229,022,806","229,022,806"
5,20250506,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"654,334,672",0,"675,602,815","675,531,255","675,602,122","675,605,749"
6,20250507,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"229,125,586",0,"224,161,542","223,690,403","223,670,115","223,670,115"
7,20250508,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"227,633,493",0,"225,753,269","188,984,579","188,983,688","188,983,688"
8,20250509,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"225,067,859",0,"221,456,281","185,293,574","185,292,315","185,292,315"
9,20250510,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,"966,711,241",0,"931,271,133","894,264,405","894,340,230","894,345,810"


### New

In [64]:
''' Prep Monthly Data '''

raw_monthly_df = chk_src_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
raw_monthly_df = raw_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
raw_monthly_df['GAP'] = raw_monthly_df['ACTUAL_SNAP'] - raw_monthly_df['TARGET_SNAP']
raw_monthly_df['%ACH'] = raw_monthly_df['ACTUAL_SNAP'] / raw_monthly_df['TARGET_SNAP'] * 100
raw_monthly_df = raw_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'AREA_TYPE', 'AREA_CD']).reset_index(drop=True)
raw_monthly_df.tail()

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
840,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910095,Si Sa Ket,9.017149e+07,8.119528e+07,8.976210e+06,111.055088
841,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910096,Surin,7.982565e+07,8.185532e+07,-2.029676e+06,97.520410
842,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910097,Ubon Ratchathani,1.794720e+08,1.251317e+08,5.434028e+07,143.426472
843,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910098,Yasothon,3.830530e+07,3.729007e+07,1.015221e+06,102.722496
844,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,P,P,Nationwide,1.039815e+10,1.077145e+10,-3.732940e+08,96.534412


In [74]:
''' 8 Region (G) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

g_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
g_monthly_df = g_monthly_df.loc[g_monthly_df['AREA_TYPE']=='G']
g_monthly_df = g_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)

g_monthly_df_display = g_monthly_df.copy()
mod_col_list = g_monthly_df_display.iloc[:, 9:12].columns.tolist()
for col in mod_col_list:
    g_monthly_df_display[col] = g_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
g_monthly_df_display['%ACH'] = g_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))

g_monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX1,BMA : West,"1,501,143,154","1,516,161,257","-15,018,104",99.01
1,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX2,BMA : East,"1,747,462,726","1,832,338,419","-84,875,693",95.37
2,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX3,East,"1,803,950,098","1,799,689,709","4,260,388",100.24
3,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX4,North,"1,133,089,170","1,239,726,001","-106,636,831",91.40
4,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX5,Northeast 1,"906,872,474","1,009,900,715","-103,028,241",89.80
5,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX6,Northeast 2,"789,107,192","908,353,043","-119,245,850",86.87
6,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX7,"Central, West","1,286,244,505","1,345,256,231","-59,011,726",95.61
7,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-28 23:53:59,20250527,G,GX8,South,"1,048,914,660","1,120,019,626","-71,104,966",93.65


In [75]:
''' 64 Province (H) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

h_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
h_monthly_df = h_monthly_df.loc[h_monthly_df['AREA_TYPE']=='H']
h_monthly_df = h_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)

h_monthly_df_display = h_monthly_df.copy()
mod_col_list = h_monthly_df_display.iloc[:, 9:12].columns.tolist()
for col in mod_col_list:
    h_monthly_df_display[col] = h_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
h_monthly_df_display['%ACH'] = h_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))

h_monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,003,"SMP : Bang Bo, Bang Sao Thong, Bang Phli","259,709,039","280,863,772","-21,154,733",92.47
1,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,004,"SMP : Mueang Samut Prakan, Phra Pradaeng, Phra...","290,010,623","311,277,438","-21,266,815",93.17
2,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,005,"NTB : Pak Kret, Bang Bua Thong, Sai Noi","212,237,696","234,068,320","-21,830,624",90.67
3,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,006,"PTT : Thanyaburi, Nong Suea, Lam Luk Ka","187,961,891","205,195,085","-17,233,195",91.60
4,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,007,"PTT : Mueang Pathum Thani, Lat Lum Kaeo, Sam K...","259,718,385","268,547,338","-8,828,953",96.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,84Z,Surat Thani,"162,452,907","181,547,013","-19,094,105",89.48
60,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,86X,"Ranong, Chumphon","71,337,601","83,093,244","-11,755,643",85.85
61,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,90Z,Songkhla,"199,598,278","223,404,504","-23,806,226",89.34
62,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 01:02:55,20250528,H,93X,"Trang, Satun, Phatthalung","124,877,967","143,031,722","-18,153,755",87.31


In [76]:
''' 96 District (HH) : Monthly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

hh_monthly_df = raw_monthly_df.loc[raw_monthly_df['TM_KEY_MTH']==v_tm_key_mth].copy()
hh_monthly_df = hh_monthly_df.loc[hh_monthly_df['AREA_TYPE']=='HH']
hh_monthly_df = hh_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)

hh_monthly_df_display = hh_monthly_df.copy()
mod_col_list = hh_monthly_df_display.iloc[:, 9:12].columns.tolist()
for col in mod_col_list:
    hh_monthly_df_display[col] = hh_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
hh_monthly_df_display['%ACH'] = hh_monthly_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))

hh_monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH
0,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,902033,Ang Thong,"25,172,953","20,269,759","4,903,194",124.19
1,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,902034,Chai Nat,"31,884,808","28,272,416","3,612,392",112.78
2,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,902035,Kanchanaburi,"53,137,724","46,509,565","6,628,159",114.25
3,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,902036,Lop Buri,"63,373,630","57,426,688","5,946,942",110.36
4,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,902037,Nakhon Pathom,"180,199,024","112,930,393","67,268,631",159.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910094,Roi Et,"86,106,639","87,444,014","-1,337,376",98.47
92,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910095,Si Sa Ket,"90,171,488","81,195,279","8,976,210",111.06
93,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910096,Surin,"79,825,646","81,855,322","-2,029,676",97.52
94,202505,All Services,B0R000100GEO,Total Revenue (Geo),2025-05-29 03:00:52,20250528,HH,910097,Ubon Ratchathani,"179,471,975","125,131,694","54,340,281",143.43


In [ ]:
        # TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        # , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        # , TARGET_SNAP, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        # , PPN_TM